In [0]:
%run "../includes/configuration"


In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
circuits_df  = spark.read.parquet(f'{processed_folder_path}/circuits')\
                        .withColumnRenamed('location', 'circuit_location')

In [0]:
races_df = spark.read.parquet(f'{processed_folder_path}/races')\
                        .withColumnRenamed('name', 'race_name')\
                        .withColumnRenamed('race_timestamp', 'race_date')

In [0]:

drivers_df = spark.read.parquet(f'{processed_folder_path}/drivers')\
                        .withColumnRenamed('name', 'driver_name')\
                        .withColumnRenamed('number', 'driver_number')\
                        .withColumnRenamed('nationality', 'driver_nationality')

In [0]:
constructor_df = spark.read.parquet(f'{processed_folder_path}/constructors')\
                        .withColumnRenamed('name', 'team')

In [0]:
results_df = spark.read.parquet(f'{processed_folder_path}/results')\
                        .withColumnRenamed('time', 'race_time')

In [0]:
race_results_df = races_df.join(results_df, races_df.race_id == results_df.race_id, 'inner') \
                          .join(drivers_df, drivers_df.driver_id == results_df.driver_id, 'inner') \
                          .join(constructor_df, constructor_df.constructor_id == results_df.constructor_id, 'inner') \
                          .join(circuits_df, circuits_df.circuit_id == races_df.circuit_id, 'inner')
                          

display(race_results_df)

#race_results_df.write.mode('overwrite').partitionBy('race_year').parquet(f'{processed_folder_path}/race_results')

In [0]:
final_race_results_df = race_results_df.select(
                              'race_year', 
                              'name',
                              'race_timestamp',
                              'location', 
                              drivers_df.name, 
                              drivers_df.number,
                              drivers_df.nationality, 
                              constructor_df.name, 
                              results_df.grid, 
                              results_df.fastest_lap,  
                              results_df.time,
                              results_df.points,
                          ).withColumn('created_date', current_timestamp())\
                          .withColumnRenamed('races_df.year', 'race_year')\
                          .withColumnRenamed('races_df.name', 'race_name')\
                          .withColumnRenamed('races_df.date', 'race_date')\
                          .withColumnRenamed('circuits_df.location', 'cicuit_location')\
                          .withColumnRenamed('drivers_df.name', 'driver_name')\
                          .withColumnRenamed('drivers_df.number', 'driver_number')\
                          .withColumnRenamed('drivers_df.nationality', 'driver_nationality')\
                          .withcolumnRenamed('results_df.time', 'race_time')
                          